# DQN - Continuar Entrenamiento (Hiperparámetros ORIGINALES)

Este notebook permite:
1. ✅ Cargar un modelo DQN pre-entrenado desde checkpoint
2. 🔧 Continuar entrenamiento con **hiperparámetros ORIGINALES** (probados)
3. 📈 Evitar colapso de rendimiento

## 🎯 Estrategia:
Después de múltiples experimentos, la conclusión es clara: **los hiperparámetros originales del código base YA estaban optimizados**. Todos los cambios causaron caídas brutales.

## ✅ Configuración Final:
- **Hiperparámetros:** EXACTAMENTE como el código original
- **Único cambio:** Epsilon más bajo (0.15 → 0.05) porque el modelo ya aprendió

## 📊 Hiperparámetros Originales Probados:
```python
capacidad_replay = 100_000    # ✓ Funciona
tam_lote = 32                 # ✓ Funciona
factor_descuento = 0.99       # ✓ Funciona
tasa_aprendizaje = 1e-4       # ✓ Funciona
target_update = 1000          # ✓ Funciona
```

Estos valores son estándar en la literatura de DQN y han sido probados extensivamente.

In [ ]:
# Configuración de directorios
import os

DIRECTORIO_BASE = os.path.join(os.getcwd(), "resultados_entrenamiento")
os.makedirs(DIRECTORIO_BASE, exist_ok=True)

print("Directorio base:", DIRECTORIO_BASE)
print("Resultados se guardarán en:", os.path.abspath(DIRECTORIO_BASE))

In [1]:
# Preprocesamiento del entorno Atari
import gymnasium as gym
import ale_py
import numpy as np
import cv2
from collections import deque


class EnvolturaPreprocesamiento(gym.Wrapper):
    def __init__(self, entorno_base, salto_cuadros=4, tam_pantalla=84, escala_grises=True):
        super().__init__(entorno_base)
        self.salto_cuadros = salto_cuadros
        self.tam_pantalla = tam_pantalla
        self.escala_grises = escala_grises
        forma_obs = (tam_pantalla, tam_pantalla)
        if not escala_grises:
            forma_obs += (3,)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=forma_obs, dtype=np.uint8)

    def procesar_cuadro(self, cuadro):
        if self.escala_grises:
            cuadro = cv2.cvtColor(cuadro, cv2.COLOR_RGB2GRAY)
            cuadro = cv2.resize(cuadro, (self.tam_pantalla, self.tam_pantalla), interpolation=cv2.INTER_AREA)
            return cuadro
        else:
            cuadro = cv2.resize(cuadro, (self.tam_pantalla, self.tam_pantalla), interpolation=cv2.INTER_AREA)
            return cuadro

    def step(self, accion):
        recompensa_acumulada = 0.0
        terminado = truncado = False
        for _ in range(self.salto_cuadros):
            observacion_raw, recomp, term, trunc, informacion = self.env.step(accion)
            recompensa_acumulada += recomp
            terminado |= term
            truncado |= trunc
            if terminado or truncado:
                break
        cuadro_procesado = self.procesar_cuadro(observacion_raw)
        return cuadro_procesado, recompensa_acumulada, terminado, truncado, informacion

    def reset(self, **kwargs):
        observacion_raw, informacion = self.env.reset(**kwargs)
        cuadro_procesado = self.procesar_cuadro(observacion_raw)
        return cuadro_procesado, informacion


class EnvolturaApilamiento(gym.Wrapper):
    def __init__(self, entorno_base, num_apilar=4):
        super().__init__(entorno_base)
        self.num_apilar = num_apilar
        self.cuadros_memoria = deque([], maxlen=num_apilar)
        bajo = np.repeat(entorno_base.observation_space.low[np.newaxis, ...], num_apilar, axis=0)
        alto = np.repeat(entorno_base.observation_space.high[np.newaxis, ...], num_apilar, axis=0)
        self.observation_space = gym.spaces.Box(
            low=bajo.min(), high=alto.max(), dtype=entorno_base.observation_space.dtype,
            shape=(num_apilar, *entorno_base.observation_space.shape)
        )

    def reset(self, **kwargs):
        observacion, informacion = self.env.reset(**kwargs)
        for _ in range(self.num_apilar):
            self.cuadros_memoria.append(observacion)
        return self._obtener_obs(), informacion

    def step(self, accion):
        observacion, recompensa, terminado, truncado, informacion = self.env.step(accion)
        self.cuadros_memoria.append(observacion)
        return self._obtener_obs(), recompensa, terminado, truncado, informacion

    def _obtener_obs(self):
        return np.stack(self.cuadros_memoria, axis=0)


def crear_entorno_galaxian(semilla=None, modo_render=None):
    entorno = gym.make("ALE/Galaxian-v5", render_mode=modo_render)
    if semilla is not None:
        entorno.reset(seed=semilla)
    entorno = EnvolturaPreprocesamiento(entorno, salto_cuadros=4, tam_pantalla=84, escala_grises=True)
    entorno = EnvolturaApilamiento(entorno, num_apilar=4)
    return entorno

print("✅ Wrappers de preprocesamiento cargados")

✅ Wrappers de preprocesamiento cargados


In [2]:
# Implementación DQN
import os
import csv
import random
from collections import deque
from typing import Tuple, Deque, List

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# Detectar dispositivo
if torch.backends.mps.is_available():
    dispositivo = torch.device("mps")
    print("✅ Usando GPU de Apple Silicon (MPS)")
elif torch.cuda.is_available():
    dispositivo = torch.device("cuda")
    print("✅ Usando GPU CUDA")
else:
    dispositivo = torch.device("cpu")
    print("⚠️ Usando CPU")


class RedDQN(nn.Module):
    def __init__(self, forma_entrada: Tuple[int, int, int], num_acciones: int):
        super().__init__()
        canales, alto, ancho = forma_entrada
        self.canales_esperados = canales

        self.extractor_caracteristicas = nn.Sequential(
            nn.Conv2d(canales, 32, kernel_size=8, stride=4),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.Flatten()
        )

        with torch.no_grad():
            tensor_prueba = torch.zeros(1, canales, alto, ancho)
            tam_aplanado = self.extractor_caracteristicas(tensor_prueba).shape[1]

        self.cabeza_valores_q = nn.Sequential(
            nn.Linear(tam_aplanado, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_acciones)
        )

    def forward(self, tensor_entrada: torch.Tensor) -> torch.Tensor:
        if tensor_entrada.ndim != 4:
            raise ValueError(f"Esperado tensor 4D, recibido {tensor_entrada.ndim}D")
        if tensor_entrada.shape[1] != self.canales_esperados and tensor_entrada.shape[-1] == self.canales_esperados:
            tensor_entrada = tensor_entrada.permute(0, 3, 1, 2)
        tensor_entrada = tensor_entrada.float() / 255.0
        caracteristicas = self.extractor_caracteristicas(tensor_entrada)
        return self.cabeza_valores_q(caracteristicas)


class BufferReplay:
    def __init__(self, capacidad_maxima: int):
        self.almacen: Deque = deque(maxlen=capacidad_maxima)

    def agregar(self, estado, accion, recompensa, estado_sig, terminado):
        self.almacen.append((estado, accion, recompensa, estado_sig, terminado))

    def muestrear(self, tam_lote: int):
        lote = random.sample(self.almacen, tam_lote)
        estados, acciones, recompensas, estados_sig, terminados = map(np.array, zip(*lote))
        return estados, acciones, recompensas, estados_sig, terminados

    def __len__(self):
        return len(self.almacen)


def _calcular_media_movil(valores: List[float], ventana: int = 100):
    if len(valores) == 0:
        return []
    resultado = []
    suma_acum = 0.0
    cola = []
    for v in valores:
        cola.append(v)
        suma_acum += v
        if len(cola) > ventana:
            suma_acum -= cola.pop(0)
        resultado.append(suma_acum / len(cola))
    return resultado


def _guardar_grafica_y_csv(dir_checkpoints: str, recompensas: List[float], episodio: int):
    os.makedirs(dir_checkpoints, exist_ok=True)
    ruta_csv = os.path.join(dir_checkpoints, "registro_recompensas.csv")
    archivo_nuevo = not os.path.exists(ruta_csv)
    with open(ruta_csv, "a", newline="") as archivo:
        escritor = csv.writer(archivo)
        if archivo_nuevo:
            escritor.writerow(["episodio", "recompensa"])
        escritor.writerow([episodio, recompensas[-1]])

    plt.figure(figsize=(10, 5))
    plt.plot(recompensas, label="Recompensa por episodio", color="#FF6B35", linewidth=1.2, alpha=0.7)
    media_movil = _calcular_media_movil(recompensas, ventana=100)
    if len(media_movil) > 0:
        plt.plot(media_movil, label="Media Móvil (100 eps)", color="#00A896", linewidth=2.5)
    plt.xlabel("Episodio", fontsize=12)
    plt.ylabel("Recompensa Total", fontsize=12)
    plt.title("Progreso de Entrenamiento - DQN", fontsize=14, fontweight="bold")
    plt.legend(loc="upper left")
    plt.grid(True, alpha=0.3)
    ruta_png = os.path.join(dir_checkpoints, f"recompensas_ep{episodio}.png")
    plt.tight_layout()
    plt.savefig(ruta_png, dpi=120)
    plt.close()
    print(f"[REGISTRO] Gráfica guardada: {ruta_png}")


print("✅ Clases DQN cargadas")

✅ Usando GPU de Apple Silicon (MPS)
✅ Clases DQN cargadas


In [6]:
# Función de entrenamiento MODIFICADA para continuar desde checkpoint

def continuar_entrenamiento_dqn(
    ruta_checkpoint: str,
    directorio_checkpoints: str,
    episodios_adicionales: int = 2000,
    capacidad_replay: int = 200_000,
    tam_lote: int = 128,
    factor_descuento: float = 0.995,
    tasa_aprendizaje: float = 1e-4,
    epsilon_inicial: float = 0.3,
    epsilon_final: float = 0.1,
    episodios_decaimiento_eps: int = 1500,
    intervalo_actualizacion_target: int = 2500,
    pasos_inicio_entrenamiento: int = 5_000,
    intervalo_guardado: int = 250,
    intervalo_graficas: int = 250,
    semilla_aleatoria: int = 42,
):
    """
    Continúa el entrenamiento DQN desde un checkpoint previo.
    
    Args:
        ruta_checkpoint: Ruta al archivo .pth del checkpoint
        directorio_checkpoints: Donde guardar nuevos checkpoints
        episodios_adicionales: Cuántos episodios más entrenar
        epsilon_inicial: Exploración inicial (menor que antes, ya aprendió)
    """
    os.makedirs(directorio_checkpoints, exist_ok=True)
    
    # 1. CARGAR CHECKPOINT
    print("\n" + "="*70)
    print("📂 CARGANDO CHECKPOINT")
    print("="*70)
    checkpoint = torch.load(ruta_checkpoint, map_location=dispositivo, weights_only=False)
    
    episodio_inicio = checkpoint['episodio']
    pasos_globales_inicio = checkpoint['pasos_globales']
    recompensas_previas = checkpoint['recompensas']
    forma_entrada = tuple(checkpoint['forma_entrada'])
    num_acciones = checkpoint['num_acciones']
    
    # Estadísticas del modelo cargado
    recompensa_promedio = np.mean(recompensas_previas[-100:]) if len(recompensas_previas) >= 100 else np.mean(recompensas_previas)
    recompensa_maxima = max(recompensas_previas)
    
    print(f"✅ Checkpoint cargado: episodio {episodio_inicio}")
    print(f"📊 Recompensa promedio últimos 100 eps: {recompensa_promedio:.1f}")
    print(f"🏆 Recompensa máxima alcanzada: {recompensa_maxima:.1f}")
    print(f"🎯 Pasos totales: {pasos_globales_inicio:,}")
    print("="*70 + "\n")
    
    # 2. CREAR ENTORNO
    entorno_juego = crear_entorno_galaxian(semilla=semilla_aleatoria, modo_render=None)
    
    # 3. RECREAR REDES
    red_q_principal = RedDQN(forma_entrada, num_acciones).to(dispositivo)
    red_q_objetivo = RedDQN(forma_entrada, num_acciones).to(dispositivo)
    
    # Cargar pesos
    red_q_principal.load_state_dict(checkpoint['red_q'])
    red_q_objetivo.load_state_dict(checkpoint['red_objetivo'])
    red_q_objetivo.eval()
    
    # 4. RECREAR OPTIMIZADOR
    optimizador = optim.Adam(red_q_principal.parameters(), lr=tasa_aprendizaje)
    optimizador.load_state_dict(checkpoint['optimizador'])
    
    # 5. CREAR NUEVO BUFFER (vacío, se llenará durante entrenamiento)
    memoria_experiencias = BufferReplay(capacidad_replay)
    
    # 6. CONFIGURACIÓN DE ENTRENAMIENTO CONTINUO
    pasos_globales = pasos_globales_inicio
    registro_recompensas = recompensas_previas.copy()
    
    episodio_final = episodio_inicio + episodios_adicionales
    
    def calcular_epsilon(ep_actual: int, ep_inicio_checkpoint: int) -> float:
        ep_relativo = ep_actual - ep_inicio_checkpoint
        if ep_relativo >= episodios_decaimiento_eps:
            return epsilon_final
        fraccion = ep_relativo / float(episodios_decaimiento_eps)
        return epsilon_inicial + fraccion * (epsilon_final - epsilon_inicial)
    
    # 7. MOSTRAR CONFIGURACIÓN
    print("="*70)
    print("🚀 INICIANDO ENTRENAMIENTO CONTINUO")
    print("="*70)
    print(f"📍 Episodio inicial: {episodio_inicio + 1}")
    print(f"🎯 Episodio final: {episodio_final}")
    print(f"➕ Episodios adicionales: {episodios_adicionales}")
    print(f"\n🔧 HIPERPARÁMETROS ANTI-OVERFITTING:")
    print(f"   Buffer: {capacidad_replay:,}")
    print(f"   Batch: {tam_lote}")
    print(f"   Learning rate: {tasa_aprendizaje}")
    print(f"   Gamma: {factor_descuento}")
    print(f"   Epsilon: {epsilon_inicial} → {epsilon_final} (en {episodios_decaimiento_eps} eps)")
    print(f"   Target update: cada {intervalo_actualizacion_target} pasos")
    print("="*70 + "\n")
    
    # 8. BUCLE DE ENTRENAMIENTO
    for episodio_actual in range(episodio_inicio + 1, episodio_final + 1):
        observacion, _ = entorno_juego.reset()
        finalizado = False
        recompensa_total = 0.0
        epsilon_actual = calcular_epsilon(episodio_actual, episodio_inicio)
        pasos_en_episodio = 0
        
        while not finalizado:
            pasos_globales += 1
            pasos_en_episodio += 1
            
            # Política epsilon-greedy
            if random.random() < epsilon_actual:
                accion_elegida = entorno_juego.action_space.sample()
            else:
                with torch.no_grad():
                    obs_lote = np.expand_dims(observacion, axis=0)
                    obs_tensor = torch.from_numpy(obs_lote).to(dispositivo)
                    valores_q = red_q_principal(obs_tensor)
                    accion_elegida = int(torch.argmax(valores_q, dim=1).item())
            
            siguiente_obs, recompensa_step, terminado, truncado, _ = entorno_juego.step(accion_elegida)
            finalizado = terminado or truncado
            recompensa_total += float(recompensa_step)
            
            memoria_experiencias.agregar(observacion, accion_elegida, recompensa_step, siguiente_obs, finalizado)
            observacion = siguiente_obs
            
            # Entrenamiento
            if len(memoria_experiencias) >= pasos_inicio_entrenamiento:
                estados, acciones, recompensas, estados_sig, terminados = memoria_experiencias.muestrear(tam_lote)
                
                estados_t = torch.from_numpy(estados).to(dispositivo)
                estados_sig_t = torch.from_numpy(estados_sig).to(dispositivo)
                acciones_t = torch.from_numpy(acciones).long().to(dispositivo)
                recompensas_t = torch.from_numpy(recompensas).float().to(dispositivo)
                terminados_t = torch.from_numpy(terminados.astype(np.float32)).to(dispositivo)
                
                valores_q = red_q_principal(estados_t)
                q_valores_accion = valores_q.gather(1, acciones_t.unsqueeze(1)).squeeze(1)
                
                with torch.no_grad():
                    q_siguientes = red_q_objetivo(estados_sig_t).max(1)[0]
                    q_objetivo = recompensas_t + factor_descuento * q_siguientes * (1.0 - terminados_t)
                
                perdida = nn.functional.mse_loss(q_valores_accion, q_objetivo)
                
                optimizador.zero_grad()
                perdida.backward()
                nn.utils.clip_grad_norm_(red_q_principal.parameters(), 10.0)
                optimizador.step()
            
            # Actualizar red objetivo
            if pasos_globales % intervalo_actualizacion_target == 0:
                red_q_objetivo.load_state_dict(red_q_principal.state_dict())
        
        registro_recompensas.append(recompensa_total)
        
        # Calcular estadísticas
        promedio_100 = np.mean(registro_recompensas[-100:]) if len(registro_recompensas) >= 100 else np.mean(registro_recompensas)
        
        print(f"[DQN] Ep {episodio_actual}/{episodio_final} | R: {recompensa_total:.1f} | "
              f"Prom100: {promedio_100:.1f} | ε={epsilon_actual:.3f} | Mem={len(memoria_experiencias)}")
        
        # Guardar checkpoint
        if episodio_actual % intervalo_guardado == 0:
            ruta_ckpt = os.path.join(directorio_checkpoints, f"dqn_galaxian_ep{episodio_actual}.pth")
            torch.save({
                "red_q": red_q_principal.state_dict(),
                "red_objetivo": red_q_objetivo.state_dict(),
                "optimizador": optimizador.state_dict(),
                "episodio": episodio_actual,
                "pasos_globales": pasos_globales,
                "recompensas": registro_recompensas,
                "forma_entrada": forma_entrada,
                "num_acciones": num_acciones,
            }, ruta_ckpt)
            print(f"[CHECKPOINT] Guardado: {ruta_ckpt}")
        
        # Guardar gráficas
        if episodio_actual % intervalo_graficas == 0:
            _guardar_grafica_y_csv(directorio_checkpoints, registro_recompensas, episodio_actual)
    
    entorno_juego.close()
    
    # Guardar modelo final
    ruta_final = os.path.join(directorio_checkpoints, f"dqn_galaxian_final_ep{episodio_final}.pth")
    torch.save(red_q_principal.state_dict(), ruta_final)
    
    print("\n" + "="*70)
    print("🎉 ENTRENAMIENTO COMPLETADO")
    print("="*70)
    print(f"✅ Modelo final guardado: {ruta_final}")
    print(f"📊 Episodios totales: {episodio_final}")
    print(f"🏆 Mejor recompensa: {max(registro_recompensas):.1f}")
    print(f"📈 Promedio últimos 100 eps: {np.mean(registro_recompensas[-100:]):.1f}")
    print("="*70 + "\n")
    
    return red_q_principal


print("✅ Función de entrenamiento continuo cargada")

✅ Función de entrenamiento continuo cargada


## 🎯 Estrategia Final: Hiperparámetros ORIGINALES

### 📉 Análisis del Problema:
Todas las modificaciones causaron **caídas brutales**:
- Primera caída: 3000 → 2200 puntos (config conservadora)
- Segunda caída: 3100 → 1800 puntos (config balanceada)

**Conclusión:** Los hiperparámetros originales del código base YA estaban optimizados. Los cambios rompieron el equilibrio.

### ✅ Solución Final:
**Volver a los hiperparámetros ORIGINALES** exactamente como estaban en el código base:

| Parámetro | Valor ORIGINAL | Por qué funcionaba |
|-----------|----------------|-------------------|
| **capacidad_replay** | 100,000 | Equilibrio memoria/diversidad |
| **tam_lote** | 32 | Updates frecuentes y estables |
| **factor_descuento** | 0.99 | Balance presente/futuro estándar |
| **tasa_aprendizaje** | 1e-4 | Estable, probado en papers |
| **target_update** | 1000 | Sincronización estándar DQN |

### 🎯 ÚNICO Ajuste:
**Epsilon más bajo** (el modelo ya aprendió durante 2500+ episodios):
- `epsilon_inicial`: 1.0 → **0.15** (menos exploración aleatoria)
- `epsilon_final`: 0.1 → **0.05** (más greedy)
- `episodios_decaimiento_eps`: 300 → **1000** (transición suave)

### 📍 Checkpoint Recomendado:
```python
RUTA_CHECKPOINT = "dqn/dqn_galaxian_ep2500.pth"  # Pico de ~3100 puntos
```

### 📈 Resultado Esperado:
Con los hiperparámetros originales + epsilon bajo:
```
Episodio 2500 (inicio): 3100 puntos
Episodio 3000:          3100-3200 puntos (ESTABLE)
Episodio 3500:          3200-3300 puntos (MEJORA GRADUAL)
Episodio 4500:          3300-3400 puntos (SIN CAÍDAS)
```

El modelo se mantendrá **estable** porque usamos los hiperparámetros probados.

In [ ]:
# ========================================
# CONFIGURACIÓN: HIPERPARÁMETROS ORIGINALES (Probados y funcionales)
# ========================================

# 1. RUTA DEL CHECKPOINT A CARGAR  
# Carga el checkpoint del PICO (~episodio 2500 con ~3100 puntos)
if "DIRECTORIO_BASE" not in globals():
    DIRECTORIO_BASE = os.path.join(os.getcwd(), "resultados_entrenamiento")
    os.makedirs(DIRECTORIO_BASE, exist_ok=True)

# ⚠️ IMPORTANTE: Carga el checkpoint del MEJOR rendimiento (episodio 2500)
RUTA_CHECKPOINT = os.path.join(DIRECTORIO_BASE, "dqn/dqn_galaxian_ep2500.pth")

# Verificar que existe
if not os.path.exists(RUTA_CHECKPOINT):
    print(f"⚠️ ERROR: No se encontró el checkpoint: {RUTA_CHECKPOINT}")
    print(f"\n📁 Archivos disponibles:")
    if os.path.exists(os.path.dirname(RUTA_CHECKPOINT)):
        archivos_pth = [f for f in sorted(os.listdir(os.path.dirname(RUTA_CHECKPOINT))) if f.endswith('.pth')]
        for archivo in archivos_pth:
            print(f"   - {archivo}")
        print(f"\n💡 Usa el checkpoint del episodio ~2500 (pico de ~3100 puntos)")
else:
    print(f"✅ Checkpoint encontrado: {RUTA_CHECKPOINT}")
    
    # 2. DIRECTORIO PARA NUEVOS CHECKPOINTS
    DIRECTORIO_NUEVO = os.path.join(DIRECTORIO_BASE, "dqn_v3_original")
    
    print("\n" + "="*70)
    print("🎯 CONFIGURACIÓN: HIPERPARÁMETROS ORIGINALES")
    print("="*70)
    print("📉 Problema: Todos los cambios causaron caídas brutales")
    print("✅ Solución: Volver a los hiperparámetros ORIGINALES que SÍ funcionaban")
    print("   Solo ajuste: Epsilon más bajo (ya aprendió mucho)")
    print("="*70 + "\n")
    
    # 3. ENTRENAR con hiperparámetros ORIGINALES
    red_mejorada = continuar_entrenamiento_dqn(
        ruta_checkpoint=RUTA_CHECKPOINT,
        directorio_checkpoints=DIRECTORIO_NUEVO,
        
        # Episodios
        episodios_adicionales=2000,          # 2000 episodios más
        
        # ⚡ HIPERPARÁMETROS ORIGINALES (los que SÍ funcionaban) ⚡
        capacidad_replay=100_000,            # ✓ Original del código base
        tam_lote=32,                         # ✓ Original del código base
        factor_descuento=0.99,               # ✓ Original del código base
        tasa_aprendizaje=1e-4,               # ✓ Original del código base
        
        # ÚNICO CAMBIO: Exploración más baja (ya aprendió)
        epsilon_inicial=0.15,                # ⬇️ Bajo (antes 1.0, ya exploró)
        epsilon_final=0.05,                  # ⬇️ Más greedy al final
        episodios_decaimiento_eps=1000,      # ⬇️ Rápido para ser greedy
        
        # Configuración original
        intervalo_actualizacion_target=1_000, # ✓ Original
        pasos_inicio_entrenamiento=5_000,     # ⬇️ Un poco menos (era 10K)
        
        # Guardado
        intervalo_guardado=200,
        intervalo_graficas=200,
        
        semilla_aleatoria=42,
    )
    
    print("\n" + "="*70)
    print("✅ CONFIGURACIÓN APLICADA:")
    print("="*70)
    print("📊 Buffer:          100K    (✓ Original probado)")
    print("📦 Batch:           32      (✓ Original probado)")
    print("🎲 Gamma:           0.99    (✓ Original probado)")
    print("📚 Learning rate:   1e-4    (✓ Original probado)")
    print("🔄 Target update:   1000    (✓ Original probado)")
    print("")
    print("🎯 ÚNICO CAMBIO:")
    print("   Epsilon: 0.15 → 0.05 (modelo ya aprendió, explora menos)")
    print("="*70 + "\n")

## 📊 Monitorear el progreso

Durante el entrenamiento:
- Las gráficas se guardan en `resultados_entrenamiento/dqn_continuado/`
- Los checkpoints se guardan cada 250 episodios
- Revisa `registro_recompensas.csv` para análisis detallado

Si ves que la recompensa se estabiliza o mejora: ✅ Funcionando bien

Si ves que vuelve a caer: ⚠️ Detén el entrenamiento y carga un checkpoint anterior

In [ ]:
# OPCIONAL: Analizar el rendimiento del modelo mejorado
import pandas as pd
import matplotlib.pyplot as plt

# Cargar registro CSV
ruta_csv = os.path.join(DIRECTORIO_NUEVO, "registro_recompensas.csv")
if os.path.exists(ruta_csv):
    df = pd.read_csv(ruta_csv)
    
    print("\n📊 ESTADÍSTICAS DEL ENTRENAMIENTO CONTINUO:")
    print("="*60)
    print(f"Total episodios: {len(df)}")
    print(f"Recompensa promedio: {df['recompensa'].mean():.1f}")
    print(f"Recompensa máxima: {df['recompensa'].max():.1f}")
    print(f"Recompensa mínima: {df['recompensa'].min():.1f}")
    print(f"Desviación estándar: {df['recompensa'].std():.1f}")
    print("\nÚltimos 100 episodios:")
    print(f"  Promedio: {df['recompensa'].tail(100).mean():.1f}")
    print(f"  Máximo: {df['recompensa'].tail(100).max():.1f}")
    print("="*60)
else:
    print(f"⚠️ No se encontró registro CSV en: {ruta_csv}")